In [ ]:
# Upload the packages
import ee 
import geemap
import os

In [ ]:
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[42.4346, 13.5959], zoom=10)
Map

In [ ]:
# Draw shapes on the interactive map using the Drawing tools & use it as ROI
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

In [ ]:
# Load a pre-computed Landsat7 composite for input.
landsat2014 = ee.Image('LANDSAT/LE7_TOA_1YEAR/2014')

# Display the roi.
Map.setCenter(13.5959, 42.4346, 10)
Map.addLayer(ee.Image().paint(roi, 0, 2), {}, 'roi')

In [ ]:
# UNSUPERVISED CLASSIFICATION (kMeans)
# Make the training dataset.
training = landsat2014.sample(**{
  'region': roi,
  'scale': 30,
  'tileScale': 8,
  'numPixels': 5000
})

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(10).train(training)

# Cluster the input using the trained clusterer.
result = landsat2014.cluster(clusterer)

# Display the clusters with random colors & clip on the ROI.
Map.addLayer(result.randomVisualizer().clip(roi), {}, 'clusters')

In [ ]:
# Select the path & define the filename (change the path with yours)
path = '/Users/...'
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'Unsupervised_Class_Landsat2014.tif')

In [ ]:
# Export the image in a local directory
geemap.ee_export_image(result, 
                       filename=filename, 
                       scale=30,
                       region=roi, 
                       file_per_band=True)